<a href="https://colab.research.google.com/github/hkhong72/big_data/blob/main/221020_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

이클립스에 Maven Project만들기
create simple Project 체크

com.adacho

hadoopMR

0.1~~ 기본셋팅

jar 기본셋팅

프로젝트생성

리눅스에서 자바1.8로 설정했기때문에

pom.xml 에서 설정바꿔야함



In [ ]:
<properties>
	<project.build.sourceEncoding>UTF-8</project.build.sourceEncoding>
	<maven.compiler.source>1.8</maven.compiler.source>
	<maven.compiler.target>1.8</maven.compiler.target>
</properties> 

<dependencies>
      <!-- https://mvnrepository.com/artifact/org.apache.hadoop/hadoop-common -->
   <dependency>
       <groupId>org.apache.hadoop</groupId>
       <artifactId>hadoop-common</artifactId>
       <version>3.2.4</version>
   </dependency>
   <!-- https://mvnrepository.com/artifact/org.apache.hadoop/hadoop-mapreduce-client-core -->
   <dependency>
       <groupId>org.apache.hadoop</groupId>
       <artifactId>hadoop-mapreduce-client-core</artifactId>
       <version>3.2.4</version>
   </dependency>

   </dependencies> #추가하기

자바패키지에 com.adacho.mapper 패키지 생성

WordcountMapper 클래스 생성

In [ ]:
package com.adacho.mapper;

import java.io.IOException;
import java.util.StringTokenizer;

import org.apache.hadoop.io.IntWritable;
import org.apache.hadoop.io.LongWritable;
import org.apache.hadoop.io.Text;
import org.apache.hadoop.mapreduce.Mapper;

public class WordcountMapper extends Mapper<LongWritable, Text, Text, IntWritable>{
	private final static IntWritable one = new IntWritable(1);
	private Text word = new Text();
	
	@Override
	protected void map(LongWritable key, Text value, Mapper<LongWritable, Text, Text, IntWritable>.Context context)
			throws IOException, InterruptedException {
		StringTokenizer strToken = new StringTokenizer(value.toString()); //default 스페이스로짜름
		while(strToken.hasMoreTokens()) {
			word.set(strToken.nextToken());
			context.write(word, one); //단어마다 1을 붙임
		}
		
	}
	
	
}


LongWritable, Text, Text, IntWritable

맵으로들어오는놈 , 맵으로나가는놈

com.apacho.reducer 패키지생성 

WordcountReducer 클래스생성

In [ ]:
package com.apacho.reducer;

import java.io.IOException;

import org.apache.hadoop.io.IntWritable;
import org.apache.hadoop.io.LongWritable;
import org.apache.hadoop.io.Text;
import org.apache.hadoop.mapreduce.Reducer;

public class WordcountReducer extends Reducer<Text, IntWritable, Text, IntWritable>{
	 private IntWritable result = new IntWritable();

	 @Override
	   protected void reduce(Text key, Iterable<IntWritable> values,
	         Reducer<Text, IntWritable, Text, IntWritable>.Context context) throws IOException, InterruptedException {
	      // TODO Auto-generated method stub
	      int sum = 0;
	      for(IntWritable value: values) {
	         sum += value.get();
	      }
	      result.set(sum);
	      context.write(key, result);
	   }
}


com.apacho.driver 패키지생성
WordCount 클래스생성

In [ ]:
package com.apacho.driver;

import org.apache.hadoop.conf.Configuration;
import org.apache.hadoop.fs.Path;
import org.apache.hadoop.io.IntWritable;
import org.apache.hadoop.io.Text;
import org.apache.hadoop.mapreduce.Job;
import org.apache.hadoop.mapreduce.lib.input.FileInputFormat;
import org.apache.hadoop.mapreduce.lib.input.TextInputFormat;
import org.apache.hadoop.mapreduce.lib.output.FileOutputFormat;
import org.apache.hadoop.mapreduce.lib.output.TextOutputFormat;

import com.adacho.mapper.WordcountMapper;
import com.apacho.reducer.WordcountReducer;

public class WordCount {
	public static void main(String[] args) throws Exception{
		Configuration conf = new Configuration();
		if(args.length !=2) {
			System.err.println("Usage: WordCount <input> <output>");
			System.exit(1);
		}
		
		Job job = Job.getInstance(conf, "WordCount");
		
		job.setJarByClass(WordCount.class); //드라이브 클래스 저장
		job.setMapperClass(WordcountMapper.class); //메퍼클래스저장
		job.setReducerClass(WordcountReducer.class); //리듀스클래스저장
		
		job.setInputFormatClass(TextInputFormat.class); //두개중에 길이가 긴 것 임포트
		job.setOutputFormatClass(TextOutputFormat.class); //두개중에 길이가 긴 것 임포트
		
		job.setMapOutputKeyClass(Text.class);
		job.setMapOutputValueClass(IntWritable.class);
		
		FileInputFormat.addInputPath(job, new Path(args[0])); //여러개에 인풋데이타가 있으면 다 줄 수 있음
		FileOutputFormat.setOutputPath(job, new Path(args[1]));
		
		job.waitForCompletion(true);
		
	}
}